In [1]:
import os
import pandas as pd
import numpy as np
from urllib.request import urlopen
import json
import matplotlib as plt
%matplotlib inline
import random
import seaborn as sns
sns.set(color_codes=True)
sns.set_context('talk',font_scale=1.2, rc={"lines.linewidth": 2.5})
sns.set_style("whitegrid")
from textblob import TextBlob,Word
os.chdir('/Volumes/jwa_drive1/git/nsf')

In [2]:
base_url = 'http://api.nsf.gov/services/v1/awards.json?rpp=25'
variables ={}
page = 1
keyword = ''
offset=1
variables['awardeeZipCode'] = ''
variables['awardeeStateCode'] = ''
variables['awardeeCity'] = 'Boston'
variables['awardeeName'] = ''
variables['perfLocation '] = ''
variables['perfCity'] = ''
variables['perfStateCode'] = ''
variables['printFields'] = 'offset,id,agency,awardeeCity,awardeeStateCode,awardeeName,date,perfCity,perfState,fundsObligatedAmt,fundProgramName,title,piFirstName,piMiddleName,piLastName,primaryProgram'
variables['offset'] = str(offset)

url = base_url

for k,v in variables.items():
    if len(v) > 0:
        url += '&' + k + '=' + v
url=url.replace(' ','')
print(url)
#http://api.nsf.gov/services/v1/awards.json?rpp=25&printFields=offset,id,agency,awardeeCity,awardeeStateCode,awardeeName,date,perfCity,perfState,fundsObligatedAmt,fundProgramName,title,piFirstName,piMiddleName,piLastName,primaryProgram&keyword=Boston


http://api.nsf.gov/services/v1/awards.json?rpp=25&awardeeCity=Boston&printFields=offset,id,agency,awardeeCity,awardeeStateCode,awardeeName,date,perfCity,perfState,fundsObligatedAmt,fundProgramName,title,piFirstName,piMiddleName,piLastName,primaryProgram&offset=1


In [3]:
df = pd.DataFrame()
while True:
    url = base_url
    for k,v in variables.items():
        if len(v) > 0:
            url += '&' + k + '=' + v
    url=url.replace(' ','')
    response = urlopen(url).read().decode('utf-8').replace('\n','')
    response_json = json.loads(response)
    if len(response_json['response']['award']) == 0:
        break
    awards_list = response_json['response']['award']
    for award_json in awards_list:
        df = df.append(award_json, ignore_index=True)
    variables['offset'] = str(int(response_json['response']['parameters']['offset'])+25)
    

In [4]:
df.shape

(3162, 13)

In [5]:
df['year'] = pd.Series(len(df['date']),index=df.index)
int_list = list(range(0, df.shape[0]))
for x in int_list:
    df.set_value(x,'year',int(str(df.ix[x]['date'])[6:]))
    df.set_value(x,'fundsObligatedAmt',float(df.ix[x]['fundsObligatedAmt']))
                 

In [6]:
# Define the aggregation calculations
aggregations = {
    'fundProgramName' : {
        'total grants' : 'count'
    },
    'year' : {
        'grants per year' : 'count'
        
    },

    'primaryProgram' : {
        'Primary Program per year' : 'count'
        
    },
    'awardeeName' : {
        'Awardee': 'count'
        
    }
}

In [7]:
grouped = df.groupby(['awardeeName','piLastName','piFirstName','year']).agg(aggregations)
grouped

primaryProgram  \
                                                            Primary Program per year   
awardeeName                     piLastName piFirstName year                            
AMERICAN METEOROLOGICAL SOC     GEER       IRA         2007                        1   
                                HOOKE      WILLIAM     2007                        1   
                                SEITTER    KEITH       2007                        1   
ARTeKCO                         Pereira    Brian       2014                        1   
Adeptrix Corporation            Bergo      Vladislav   2013                        1   
                                                       2015                        1   
American Meteorological Society Brey       James       2005                        1   
                                                       2010                        1   
                                                       2011                        1   
                                Eosco      Gina        2007                        1   
                                                       2011                        1   
                                Fisher     Genene      2004                        1   
                                                       2005                        1   
                                                       2007                        1   
                                Geer       Ira         1994                        2   
                                                       1995                        1   
                                                       1997                        1   
                                                       1998                        2   
                                                       1999                        2   
                                                       2001                        1   
                                                       2002                        2   
                                Greenfield Richard     2003                        1   
                                Hallgren   Richard     1991                        1   
                                                       1992                        2   
                                                       1993                        2   
                                                       1994                        3   
                                                       1995                        2   
                                                       1996                        5   
                                                       1998                        2   
                                Higgins    Paul        2009                        1   
...                                                                              ...   
Trustees of Boston University   Ziegler    Lawrence    1993                        1   
                                                       1997                        1   
                                                       2003                        1   
                                                       2012                        1   
                                Zimmerman  George      1992                        1   
                                                       1994                        1   
                                Zlateva    Stoyanka    2000                        1   
                                Zook       Douglas     1992                        1   
                                                       2002                        1   
                                                       2012                        1   
Tufts Medical Center, Inc       Peattie    Robert      2013                        2   
Vining Benjamin R               Vining     Benjamin    2006                        1   
Walsh Brian M         